In [4]:
from scitran_client import ScitranClient, query, Projects, Acquisitions, Sessions, Files
client = ScitranClient('scitran')

Found an existing token for this instance.
The existing token has been refreshed.


We can assemble queries by building filters like below. We're looking for sessions in the project called `vwfa`, so we query for sessions and filter by the associated project. After we find the sessions we're interested in, we query for T1-weighted nifti files that are associated with those sessions.

In [7]:
sessions = client.search(query(Sessions).filter(Projects.label.match('vwfa')))

files = client.search(query(Files).filter(
    Files.type.match('nifti'),
    Acquisitions.measurement.match('anatomy_t1w'),
    Acquisitions.session.in_(session['_id'] for session in sessions),
))

In [8]:
# Let's analyze the first file.
example_file = files[0]

import os
# fsl-bet looks for files in the nifti subdirectory
nifti_dir = os.path.join(client.gear_in_dir, 'nifti')
if not os.path.exists(nifti_dir):
    os.mkdir(nifti_dir)
example_file_path = client.download_all_file_search_results([example_file], dest_dir=nifti_dir)[0]

In [9]:
session_id = example_file['_source']['acquisition']['session']

# We let fsl-bet find the input file by having an empty string for a command.
client.run_gear_and_upload_analysis('testing fsl-bet local run', 'scitran/fsl-bet', session_id, '')

Running container scitran/fsl-bet on with input /Users/carlos/Downloads/input and output /Users/carlos/Downloads/output
[scitran/fsl-bet]  Initiated
[scitran/fsl-bet]  Running bet2 on 11353_3_1.nii.gz
[scitran/fsl-bet]  Wrote: 11353_3_1_brain-extracted.nii.gz
[scitran/fsl-bet]  generated /flywheel/v0/output/.metadata.json
[scitran/fsl-bet]  Done!
Docker container finished with exit code 0
Uploading results to collection with id 56e9d386ddea7f915e81f704.
Uploaded analysis has ID 57e6cbdf7667550013270805. Server responded with 200.


In [16]:
import nibabel as nib
import scipy.misc
img = nib.load(example_file_path)
scipy.misc.toimage(img.get_data()[100, :, :]).save('before.png')

output_file_path = os.path.join(client.gear_out_dir, '11353_3_1_brain-extracted.nii.gz')
img = nib.load(output_file_path)
scipy.misc.toimage(img.get_data()[100, :, :]).save('after.png')

![](before.png) ![](after.png)